In [26]:
!pip install --upgrade openai wikipedia-api serpapi requests gtts moviepy onnx onnxruntime huggingface_hub streamlit amadeus


In [27]:
!pip install --upgrade langchain-openai
!pip install --upgrade google-search-results

!pip install --upgrade moviepy


In [28]:
from serpapi import GoogleSearch
print("✅ GoogleSearch is imported successfully!")


✅ GoogleSearch is imported successfully!


In [29]:
import openai
import wikipediaapi
import serpapi
import requests
import gtts
import moviepy
import onnx
import onnxruntime
import huggingface_hub
import streamlit
import amadeus

print("✅ All libraries are installed successfully!")


✅ All libraries are installed successfully!


In [30]:

# Create necessary files
!touch app.py travel_story.py config.py utils.py


# **Write config.py (Store API Keys)**

In [42]:
%%writefile config.py
import os
from amadeus import Client
from langchain_openai.chat_models import ChatOpenAI

# Set up API keys
os.environ["OPENAI_API_KEY"] = "YOUR_TOKEN"
os.environ["AMADEUS_API_KEY"] = "YOUR_TOKEN"
os.environ["AMADEUS_API_SECRET"] = "YOUR_TOKEN"
google_maps_api_key = "YOUR_TOKEN"
weather_api_key = "YOUR_TOKEN"
serpapi_key = "YOUR_TOKEN"
huggingface_token = "YOUR_TOKEN"

amadeus_api_key = os.environ["AMADEUS_API_KEY"]
amadeus_api_secret = os.environ["AMADEUS_API_SECRET"]


# ✅ Initialize Amadeus API client
amadeus = Client(client_id=os.getenv("AMADEUS_API_KEY"), client_secret=os.getenv("AMADEUS_API_SECRET"))

# ✅ Initialize OpenAI GPT-4 Model
llm = ChatOpenAI(model="gpt-4", temperature=0, openai_api_key=os.getenv("OPENAI_API_KEY"))


Overwriting config.py


In [43]:
try:
    from config import google_maps_api_key, serpapi_key, weather_api_key
    print("✅ Successfully imported API keys from config.py")
    print(f"Google Maps API Key: {google_maps_api_key[:5]}******")  # Hide full key for security
    print(f"SerpAPI Key: {serpapi_key[:5]}******")
    print(f"Weather API Key: {weather_api_key[:5]}******")
except ModuleNotFoundError:
    print("❌ ERROR: 'config.py' not found. Ensure the file exists in the same directory.")
except ImportError:
    print("❌ ERROR: Could not import variables from 'config.py'. Check variable names.")


✅ Successfully imported API keys from config.py
Google Maps API Key: AIzaS******
SerpAPI Key: c71f9******
Weather API Key: 8af06******


# **Write utils.py (Helper Functions for API Calls)**

In [52]:
%%writefile utils.py

import requests
from config import google_maps_api_key, weather_api_key, amadeus_api_key, amadeus_api_secret

# ✅ Function to fetch latitude & longitude
def get_lat_lng(location):
    url = "https://maps.googleapis.com/maps/api/geocode/json"
    params = {"address": location, "key": google_maps_api_key}
    response = requests.get(url, params=params).json()
    if "results" in response and response["results"]:
        location_data = response["results"][0]["geometry"]["location"]
        return location_data["lat"], location_data["lng"]
    return None, None

# ✅ Function to fetch top tourist attractions dynamically
def fetch_tourist_attractions(location, top_n=5):
    lat, lng = get_lat_lng(location)
    if not lat or not lng:
        return "Could not determine the exact location."

    url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    params = {"location": f"{lat},{lng}", "radius": 10000, "type": "tourist_attraction", "key": google_maps_api_key}

    response = requests.get(url, params=params).json()
    if "results" in response:
        return "\n".join(
            [f"🏛 {t['name']} ({t.get('rating', 'No rating')}⭐)" for t in response["results"][:top_n]]
        )
    return "No tourist attractions found."



# ✅ Function to fetch restaurants dynamically
def fetch_restaurants(location, purpose, top_n=5):
    lat, lng = get_lat_lng(location)
    if not lat or not lng:
        return "Could not determine the exact location."

    keyword = {
        "Leisure": "casual dining",
        "Business": "fine dining",
        "Family": "family-friendly",
        "Adventure": "unique cuisine",
        "Romantic": "romantic restaurant"
    }.get(purpose, "restaurant")

    url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    params = {"location": f"{lat},{lng}", "radius": 5000, "type": "restaurant", "keyword": keyword, "key": google_maps_api_key}

    response = requests.get(url, params=params).json()
    if "results" in response:
        return "\n".join(
            [f"🍽 {r['name']} ({r.get('rating', 'No rating')}⭐)" for r in response["results"][:top_n]]
        )
    return "No restaurants found."

# ✅ Function to fetch real-time weather details
def fetch_weather(city):
    url = "http://api.openweathermap.org/data/2.5/weather"
    params = {"q": city, "appid": weather_api_key, "units": "metric"}
    response = requests.get(url, params=params).json()
    if "weather" in response and "main" in response:
        return f"🌤 {response['weather'][0]['description'].capitalize()}, {response['main']['temp']}°C"
    return "Weather data not available."

# ✅ Function to fetch 7-day weather forecast
def fetch_weather_forecast(city):
    url = "http://api.openweathermap.org/data/2.5/forecast"
    params = {"q": city, "appid": weather_api_key, "units": "metric"}
    response = requests.get(url, params=params).json()

    if "list" in response:
        forecast_data = response["list"][:7]
        forecast = "\n".join(
            [f"📅 {entry['dt_txt']} - {entry['weather'][0]['description'].capitalize()}, {entry['main']['temp']}°C"
             for entry in forecast_data]
        )
        return forecast
    return "Weather forecast data not available."


# ✅ Function to fetch flight details with improved error handling
def fetch_flight_details(origin, destination, departure_date):
    try:
        response = requests.get(f"https://api.flightapi.com/{origin}/{destination}/{departure_date}", headers={"API-Key": amadeus_api_key})
        if response.status_code == 200:
            return response.json()
        return "Flight details unavailable. Please try again later."
    except Exception as e:
        return f"Error retrieving flight details: {str(e)}"

# ✅ Function to fetch a destination image with fallback options
def fetch_destination_image(destination):
    image_sources = [
        f"https://source.unsplash.com/600x400/?{destination}",
        f"https://source.unsplash.com/600x400/?travel,{destination}"
    ]

    for url in image_sources:
        response = requests.get(url, stream=True)
        if response.status_code == 200:
            return url
    return "Image unavailable. Please try again later."

# ✅ Function to fetch nearby hotels dynamically
def fetch_hotels(location, top_n=5):
    lat, lng = get_lat_lng(location)
    if not lat or not lng:
        return "Could not determine the exact location."

    url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    params = {"location": f"{lat},{lng}", "radius": 5000, "type": "lodging", "key": google_maps_api_key}

    response = requests.get(url, params=params).json()
    if "results" in response:
        return "\n".join(
            [f"🏨 {h['name']} ({h.get('rating', 'No rating')}⭐)" for h in response["results"][:top_n]]
        )
    return "No hotels found."

# ✅ Function to fetch live currency exchange rate
def fetch_currency_exchange_rate(base_currency, target_currency):
    url = f"https://api.exchangerate-api.com/v4/latest/{base_currency}"
    response = requests.get(url).json()
    if "rates" in response and target_currency in response["rates"]:
        return f"💰 1 {base_currency} = {response['rates'][target_currency]} {target_currency}"
    return "Exchange rate data not available."

# ✅ Function to fetch top travel news & insights
def fetch_travel_news():
    url = "https://serpapi.com/search.json"
    params = {"q": "latest travel trends", "api_key": google_maps_api_key}
    response = requests.get(url, params=params).json()

    if "organic_results" in response:
        return "\n".join(
            [f"📰 {article['title']} - {article['link']}" for article in response["organic_results"][:5]]
        )
    return "No travel news available."



Overwriting utils.py


# **Write travel_story.py (AI-Powered Travel Story & Video Generation)**

In [53]:
%%writefile travel_story.py

import requests
import wikipediaapi
from gtts import gTTS
from moviepy import *
from config import llm
from utils import fetch_weather, fetch_tourist_attractions, fetch_flight_details, fetch_restaurants, fetch_hotels, fetch_currency_exchange_rate, fetch_travel_news


# ✅ Wikipedia API Setup
wiki = wikipediaapi.Wikipedia(user_agent="WanderTales/1.0", language="en")

# ✅ Function to fetch Wikipedia summary dynamically
def get_wikipedia_summary(place):
    page = wiki.page(place)
    return page.summary[:500] if page.exists() else "No Wikipedia summary found."

# ✅ Function to generate a travel story based on purpose and dates
def generate_travel_story(destination, purpose, start_date, end_date):
    wikipedia_info = get_wikipedia_summary(destination)

    purpose_templates = {
        "leisure": f"You are about to explore {destination} on a relaxing leisure trip from {start_date} to {end_date}. Describe your experiences as you visit famous landmarks, stroll through parks, and enjoy the peaceful atmosphere.",
        "food": f"As a food lover, you're in {destination} from {start_date} to {end_date} to explore its delicious street food, high-end restaurants, and unique local flavors. Describe the dishes you'll taste, the bustling food markets, and the famous cafés you’ll visit.",
        "adventure": f"You're visiting {destination} for an adrenaline-filled adventure from {start_date} to {end_date}. Describe thrilling activities such as hiking, surfing, skydiving, and other outdoor experiences in the area.",
        "business": f"You're traveling to {destination} for a business trip from {start_date} to {end_date}. Describe your meetings, networking events, and the city's corporate atmosphere. Also, mention any work-life balance experiences, like after-hours dining or sightseeing.",
        "romantic": f"You're in {destination} for a romantic getaway from {start_date} to {end_date}. Describe the intimate dinners, scenic walks, and breathtaking sunset views you'll experience with your partner.",
        "spiritual": f"You're visiting {destination} for a spiritual retreat from {start_date} to {end_date}. Describe the meditation spots, temples, churches, and peaceful landscapes where you'll find tranquility and reflection.",
        "family": f"You're on a family trip to {destination} from {start_date} to {end_date}, filled with fun and bonding moments. Describe the amusement parks, kid-friendly attractions, and the joy of exploring new places together."
    }

    purpose_prompt = purpose_templates.get(purpose, purpose_templates["leisure"])

    full_prompt = f"""
    {purpose_prompt}

    Be immersive, engaging, and detailed. Use vivid descriptions and include unique aspects of {destination}.

    Wikipedia Summary: {wikipedia_info}

    Travel Story:
    """

    try:
        response = llm.invoke(full_prompt)
        return response.content if hasattr(response, 'content') else str(response)
    except Exception as e:
        return f"Error: OpenAI API call failed. Check your model name and API key: {str(e)}"

# ✅ Function to generate a voice-over for the travel story
def generate_voiceover(text, output_audio="travel_narration.mp3"):
    try:
        tts = gTTS(text=text, lang="en", slow=False)
        tts.save(output_audio)
        return output_audio
    except Exception as e:
        return f"Error generating voiceover: {str(e)}"

# ✅ Function to create a dynamic travel video with AI narration
def create_travel_video(image_urls, narration_audio, output_video="travel_story.mp4"):
    try:
        if not os.path.exists(narration_audio):
            raise FileNotFoundError(f"Audio file '{narration_audio}' not found.")

        audio_clip = AudioFileClip(narration_audio)
        image_duration = max(audio_clip.duration / max(len(image_urls), 1), 2)  # Ensure minimum duration per image

        image_clips = []
        for url in image_urls:
            try:
                response = requests.get(url, stream=True)
                if response.status_code == 200:
                    image_clips.append(ImageClip(response.raw, duration=image_duration).set_fps(24).resize(1.05))
            except Exception as img_err:
                print(f"Skipping image due to error: {img_err}")

        if not image_clips:
            raise ValueError("No valid images retrieved for video generation.")

        video_clip = concatenate_videoclips(image_clips, method="compose").set_audio(audio_clip)
        video_clip.write_videofile(output_video, codec="libx264", fps=24, audio_codec="aac")

        return output_video
    except Exception as e:
        return f"Error creating video: {str(e)}"


# ✅ Testing the script (Example Execution)
if __name__ == "__main__":
    destination = "Paris"
    purpose = "Leisure"
    start_date = "2025-04-10"
    end_date = "2025-04-20"

    print("🔄 Fetching data and generating travel story...")
    travel_story_text = generate_travel_story(destination, purpose, start_date, end_date)
    print(f"📖 Travel Story:\n{travel_story_text}")

    print("🎤 Generating voiceover...")
    narration_file = generate_voiceover(travel_story_text)

    print("🎥 Creating travel video...")
    travel_images = ["https://source.unsplash.com/600x400/?Paris", "https://source.unsplash.com/600x400/?EiffelTower"]
    travel_video = create_travel_video(travel_images, narration_file)

    print(f"✅ Travel video saved as {travel_video}")


Overwriting travel_story.py


# **Write app.py (Streamlit UI)**

In [54]:
%%writefile app.py

import streamlit as st
from travel_story import generate_travel_story, generate_voiceover, create_travel_video  # Correct import
from utils import fetch_weather, fetch_tourist_attractions, fetch_flight_details, fetch_restaurants, fetch_hotels
import requests
from PIL import Image
from io import BytesIO

st.set_page_config(page_title="✈️ AI Travel Planner", layout="wide")

st.title("✈️ AI Travel Planner 🏨")

st.sidebar.title("Plan Your Trip 🗺")
destination = st.sidebar.text_input("Enter Destination", "Paris")
start_date = st.sidebar.date_input("Start Date")
end_date = st.sidebar.date_input("End Date")
purpose = st.sidebar.selectbox("Purpose of Visit", ["Leisure", "Business", "Adventure", "Romantic", "Family"])

if st.sidebar.button("Generate Travel Story"):
    with st.spinner("🔄 Generating AI travel story..."):
        try:
            travel_story_text = generate_travel_story(destination, purpose, start_date, end_date)
        except AttributeError as e:
            st.error("⚠️ AI generation failed: Possible incorrect OpenAI API usage. Check API setup.")
            travel_story_text = "❌ Error generating AI travel story. Please try again later."

    st.subheader("📖 Your AI-Generated Travel Story")
    st.write(travel_story_text)

    st.subheader("🌦 Weather Forecast")
    st.write(fetch_weather(destination))

    st.subheader("🏛 Tourist Attractions")
    st.write(fetch_tourist_attractions(destination))

    st.subheader("🍽 Recommended Restaurants")
    st.write(fetch_restaurants(destination, purpose))

    st.subheader("🏨 Hotel Recommendations")
    st.write(fetch_hotels(destination))

    st.subheader("✈️ Flight Details")
    st.write(fetch_flight_details("JFK", destination, start_date))

    st.subheader("🖼 View Destination Image")
    image_url = f"https://source.unsplash.com/600x400/?{destination}"  # Fetch image dynamically
    response = requests.get(image_url)
    if response.status_code == 200:
        image = Image.open(BytesIO(response.content))
        st.image(image, caption=f"A view of {destination}", use_column_width=True)
    else:
        st.warning("❌ Unable to fetch image. Try again later.")

    st.subheader("🎤 Generating Voiceover...")
    narration_audio = generate_voiceover(travel_story_text)
    st.audio(narration_audio)

    st.subheader("🎥 Creating AI Travel Video...")
    travel_images = [image_url]
    travel_video = create_travel_video(travel_images, narration_audio)
    st.video(travel_video)

    st.success("✅ Travel Story & AI Narration Generated Successfully!")

Overwriting app.py


In [56]:
%%writefile verify_imports.py

import os

# ✅ Check if all necessary files exist
required_files = ["config.py", "utils.py", "travel_story.py", "app.py"]
missing_files = [file for file in required_files if not os.path.exists(file)]

if missing_files:
    print(f"❌ ERROR: Missing files: {missing_files}. Ensure all required files are present.")
else:
    print("✅ All necessary files exist.")

# ✅ Verify `config.py` imports
try:
    from config import google_maps_api_key, serpapi_key, weather_api_key
    print("✅ Successfully imported API keys from config.py")
    print(f"Google Maps API Key: {google_maps_api_key[:5]}******")
    print(f"SerpAPI Key: {serpapi_key[:5]}******")
    print(f"Weather API Key: {weather_api_key[:5]}******")
except ModuleNotFoundError:
    print("❌ ERROR: 'config.py' not found.")
except ImportError:
    print("❌ ERROR: Could not import variables from 'config.py'.")

# ✅ Verify `utils.py` imports
try:
    from utils import get_lat_lng, fetch_restaurants, fetch_weather
    print("✅ Successfully imported functions from utils.py")
    print(f"get_lat_lng function exists: {callable(get_lat_lng)}")
    print(f"fetch_restaurants function exists: {callable(fetch_restaurants)}")
    print(f"fetch_weather function exists: {callable(fetch_weather)}")
except ModuleNotFoundError:
    print("❌ ERROR: 'utils.py' not found.")
except ImportError:
    print("❌ ERROR: Could not import functions from 'utils.py'.")

# ✅ Verify `purpose.py` imports
try:
    from travel_story import generate_travel_story, generate_voiceover, create_travel_video
    print("✅ Successfully imported functions from purpose.py")
    print(f"generate_travel_story function exists: {callable(generate_travel_story)}")
    print(f"generate_voiceover function exists: {callable(generate_voiceover)}")
    print(f"create_travel_video function exists: {callable(create_travel_video)}")
except ModuleNotFoundError:
    print("❌ ERROR: 'purpose.py' not found.")
except ImportError:
    print("❌ ERROR: Could not import functions from 'purpose.py'.")

# ✅ Verify `app.py` existence
if os.path.exists("app.py"):
    print("✅ 'app.py' exists and is ready to run.")
else:
    print("❌ ERROR: 'app.py' is missing.")


Overwriting verify_imports.py


In [57]:
!python verify_imports.py


✅ All necessary files exist.
✅ Successfully imported API keys from config.py
Google Maps API Key: AIzaS******
SerpAPI Key: c71f9******
Weather API Key: 8af06******
✅ Successfully imported functions from utils.py
get_lat_lng function exists: True
fetch_restaurants function exists: True
fetch_weather function exists: True
✅ Successfully imported functions from purpose.py
generate_travel_story function exists: True
generate_voiceover function exists: True
create_travel_video function exists: True
✅ 'app.py' exists and is ready to run.


In [58]:
!pip install pyngrok

from pyngrok import ngrok
!kill $(pgrep -f ngrok)

# Run Streamlit app
!streamlit run app.py &>/content/logs.txt &
# Set up ngrok
!ngrok authtoken 2suOuuxs3zjz3pWQWDl9dQZPTLR_5FKMVwXpnfPJDKgESoGpK

public_url = ngrok.connect(8501)
print(f"Public URL: {public_url}")

^C
Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml
Public URL: NgrokTunnel: "https://9112-34-106-31-156.ngrok-free.app" -> "http://localhost:8501"


In [39]:
import os
print(os.listdir())  # List all files in the working directory


['.config', 'verify_imports.py', 'travel_story.py', 'logs.txt', 'utils.py', 'app.py', 'config.py', '__pycache__', 'travel_narration.mp3', 'sample_data']


In [41]:
# Import the necessary function
from travel_story import generate_travel_story

# ✅ Test AI storytelling generation
print("\n📖 AI-Generated Travel Story:")
travel_story = generate_travel_story("Paris", "Leisure", "2025-04-10", "2025-04-20")
print(travel_story)



📖 AI-Generated Travel Story:


AttributeError: 'AIMessage' object has no attribute 'strip'

In [ ]:
from travel_story import generate_travel_story, generate_voiceover, create_travel_video

generate_travel_story("Paris", "Leisure", "2025-04-10", "2025-04-20")

In [ ]:
from travel_story import generate_travel_story, generate_voiceover, create_travel_video
from utils import fetch_weather, fetch_flight_details, fetch_hotels, fetch_tourist_attractions

# ✅ Test real-time data retrieval
print("🌦 Weather Data:")
print(fetch_weather("Hyderabad"))

print("\n✈️ Flight Details:")
print(fetch_flight_details("JFK", "CDG", "2025-04-10"))

print("\n🏨 Hotel Recommendations:")
print(fetch_hotels("Hyderabad"))

print("\n🏛 Tourist Attractions:")
print(fetch_tourist_attractions("Paris"))


# ✅ Test AI storytelling generation
print("\n📖 AI-Generated Travel Story:")
travel_story = generate_travel_story("Paris", "Leisure", "2025-04-10", "2025-04-20")
print(travel_story)

# ✅ Test AI voice narration
print("\n🎤 Generating AI Voiceover...")
narration_audio = generate_voiceover(travel_story)
print(f"🔊 Audio file generated: {narration_audio}")

# ✅ Test AI video generation
print("\n🎥 Creating AI Travel Video...")
travel_images = ["https://source.unsplash.com/600x400/?Paris", "https://source.unsplash.com/600x400/?EiffelTower"]
travel_video = create_travel_video(travel_images, narration_audio)
print(f"🎬 Video file generated: {travel_video}")
